In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import neurokit2 as nk
import json

from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline

from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFECV, RFE

from Participant import Participant
from feature_extraction import skt_features
from helper import extract_segments, extract_features

In [2]:
### Initialize participant IDs
all_participant_ids = np.arange(1, 46)
# remove participant 5 due to tehcnical issues (incomplete experiment)
all_participant_ids = all_participant_ids[all_participant_ids != 5]
# remove participant 28 due to missing timestamps
all_participant_ids = all_participant_ids[all_participant_ids != 28]
# remove participant 19 due to technical issues (problematic physiological data)
all_participant_ids = all_participant_ids[all_participant_ids != 19]

In [3]:
classifiers = {}

In [4]:
### Window parameters: Signal to Features
delta_start = -5
duration = 24

In [25]:
# ADDED: Global trackers for minimal segment lengths across all participants
global_min_ibi_len = float('inf')
global_min_eda_len = float('inf')
global_min_skt_len = float('inf')

for pid in all_participant_ids:
    ### Initialize storage for participant-sepcific information 
    p = Participant(participantID=pid, rawData=True)
        
    # Store counts per condition *for this participant*
    condition_segment_counts = {} 
        
    for condition in {'audio+image', 'image-only', 'audio-only'}:
        ### Initialize storage for feature matrix and labels
        X = []
        y = []

        ### Storage for segments of each physiological signal
        ### Lists of dictionaries
        ibi_segments = []
        eda_segments = []
        skt_segments = []

        ### Iterate over blocks of the specified condition
        condition_mask = p.markers['condition'] == condition
        ## Drop one random sample of each target emotion for 'audio-only' condition
        if condition == 'audio-only':
            audio_only_df = p.markers[p.markers['condition'] == 'audio-only'].copy()
            target_emotions = ['Happiness', 'Anger', 'Sadness']
            filtered_audio_only_df = audio_only_df[audio_only_df['target_emotion'].isin(target_emotions)] 
            try:
                sampled_rows = filtered_audio_only_df.groupby('target_emotion').sample(n=1, random_state=42)
                remaining_rows = audio_only_df.drop(sampled_rows.index)
            except ValueError:
                remaining_rows = audio_only_df # If sampling fails, use all rows
        else:
            remaining_rows = p.markers[condition_mask]
            
        for block_index in remaining_rows.index:

            ### Get onsets depending on condition
            if condition == 'audio-only':
                audio_onset = p.markers['audio_onset'].iloc[block_index]
                # 60 second trials with 10 second intervals between segments
                # Intervals have the same duration as an image presentation
                onsets = np.arange(audio_onset, audio_onset + 60000, 10000)
            else:
                onsets = json.loads(p.markers.iloc[block_index]['image_onsets'])

            ### Extract segments for each onset
            for onset in onsets:
                timestamp = onset / 1000  # Convert milliseconds to seconds
                segment = p.get_rawData_from_timestamp(timestamp, delta_start, duration)
                ibi_segments.append(segment['systolicPeaks'])
                eda_segments.append(segment['eda'])
                skt_segments.append(segment['skt'])
                y.append(p.markers.loc[block_index, 'target_emotion'])

        # --- ADDED: NEW ASSERTION 1 (Segments are not empty) & MIN LENGTH UPDATE ---
        # Only run if segments were actually collected
        if y: # Checks if list is not empty
            assert all(len(s) > 0 for s in ibi_segments), f"Found empty IBI segment for {pid} ({condition})"
            assert all(len(s) > 0 for s in eda_segments), f"Found empty EDA segment for {pid} ({condition})"
            assert all(len(s) > 0 for s in skt_segments), f"Found empty SKT segment for {pid} ({condition})"
            
            # Update global minimums
            global_min_ibi_len = min(global_min_ibi_len, min(len(s) for s in ibi_segments))
            global_min_eda_len = min(global_min_eda_len, min(len(s) for s in eda_segments))
            global_min_skt_len = min(global_min_skt_len, min(len(s) for s in skt_segments))

        # --- EXISTING: ASSERTION 2 (Internal Consistency) ---
        n_segments = len(y)
        assert len(ibi_segments) == n_segments and len(eda_segments) == n_segments and len(skt_segments) == n_segments, \
            f"Data mismatch for {pid} ({condition}): y={n_segments}, ibi={len(ibi_segments)}, eda={len(eda_segments)}, skt={len(skt_segments)}"
        
        # Store count for this condition (for Assertion 3)
        condition_segment_counts[condition] = n_segments
    
    # --- EXISTING: ASSERTION 3 (Cross-Condition Consistency for this Participant) ---
    counts = list(condition_segment_counts.values())
    assert len(counts) == 3 and all(c == counts[0] for c in counts), \
        f"FAIL {pid}: Mismatch in segment counts across conditions: {condition_segment_counts}"
    print(f"SUCCESS {pid}: All conditions have {counts[0]} segments.")

# --- ADDED: Final Print of Global Mins ---
print("\n" + "="*50)
print("--- Global Segment Length Report ---")
print(f"Minimal IBI segment length found: {global_min_ibi_len if global_min_ibi_len != float('inf') else 'N/A'}")
print(f"Minimal EDA segment length found: {global_min_eda_len if global_min_eda_len != float('inf') else 'N/A'}")
print(f"Minimal SKT segment length found: {global_min_skt_len if global_min_skt_len != float('inf') else 'N/A'}")

SUCCESS 1: All conditions have 54 segments.
SUCCESS 2: All conditions have 54 segments.
SUCCESS 3: All conditions have 54 segments.
SUCCESS 4: All conditions have 54 segments.
SUCCESS 6: All conditions have 54 segments.
SUCCESS 7: All conditions have 54 segments.
SUCCESS 8: All conditions have 54 segments.
SUCCESS 9: All conditions have 54 segments.
SUCCESS 10: All conditions have 54 segments.
SUCCESS 11: All conditions have 54 segments.
SUCCESS 12: All conditions have 54 segments.
SUCCESS 13: All conditions have 54 segments.
SUCCESS 14: All conditions have 54 segments.
SUCCESS 15: All conditions have 54 segments.
SUCCESS 16: All conditions have 54 segments.
SUCCESS 17: All conditions have 54 segments.
SUCCESS 18: All conditions have 54 segments.
SUCCESS 20: All conditions have 54 segments.
SUCCESS 21: All conditions have 54 segments.
SUCCESS 22: All conditions have 54 segments.
SUCCESS 23: All conditions have 54 segments.
SUCCESS 24: All conditions have 54 segments.
SUCCESS 25: All co

In [80]:
X = extract_features(ibi_segments, eda_segments, skt_segments)
X = pd.DataFrame(X)

# Some features are only defined for segments of certain minimal lengths
# X could contain NaN values in columns like SampEn (1 value from 54)
X = X.replace([np.inf, -np.inf], np.nan)
X = X.drop(columns=X.columns[X.isna().all()])

preprocessor = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('remove_constant', VarianceThreshold())
])
preprocessor.set_output(transform="pandas")
X_processed = preprocessor.fit_transform(X)

c:\Users\Mario Tokumori\.conda\envs\handeln2025\lib\site-packages\neurokit2\eda\eda_clean.py:105: NeuroKitWarning: EDA signal is sampled at very low frequency. Skipping filtering.
  warn(
c:\Users\Mario Tokumori\.conda\envs\handeln2025\lib\site-packages\neurokit2\epochs\eventrelated_utils.py:37: NeuroKitWarning: eda_eventrelated(): The duration of your epochs seems quite long. You might want to use eda_intervalrelated().
  warn(
c:\Users\Mario Tokumori\.conda\envs\handeln2025\lib\site-packages\neurokit2\eda\eda_clean.py:105: NeuroKitWarning: EDA signal is sampled at very low frequency. Skipping filtering.
  warn(
c:\Users\Mario Tokumori\.conda\envs\handeln2025\lib\site-packages\neurokit2\epochs\eventrelated_utils.py:37: NeuroKitWarning: eda_eventrelated(): The duration of your epochs seems quite long. You might want to use eda_intervalrelated().
  warn(
c:\Users\Mario Tokumori\.conda\envs\handeln2025\lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:529: NeuroKitWarning: DFA_alpha2 rela

In [ ]:
### Selects the optimal number of features based on Leave-One-Out cross-validation, 
### Ideal for small datasets to prevent overfitting
rfecv_with_forest = RFECV(RandomForestClassifier(), step=1, cv=LeaveOneOut(), n_jobs=-1).fit(X_processed, y)

In [83]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# We create pipelines to test everything properly
# We *must* scale data for SVM, KNN, and NN.
# GaussianNB should NOT be scaled, as it expects Gaussian-distributed data.

# --- 1. SVM Tuning ---
pipe_svm = Pipeline([
    ('scale', StandardScaler()),
    ('svm', SVC(random_state=42))
])
# We tune 'C' (regularization) and 'kernel'
param_svm = {
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf']
}
grid_svm = GridSearchCV(pipe_svm, param_svm, cv=LeaveOneOut(), scoring='accuracy', n_jobs=-1)
grid_svm.fit(X_processed, y)
print(f"\n--- SVM ---")
print(f"Best Score: {grid_svm.best_score_:.4f}")
print(f"Best Params: {grid_svm.best_params_}")

# --- 2. KNN Tuning ---
pipe_knn = Pipeline([
    ('scale', StandardScaler()),
    ('knn', KNeighborsClassifier())
])
# We tune 'n_neighbors' (the 'k' value)
param_knn = {
    'knn__n_neighbors': [3, 5, 7, 9] # Small, odd numbers
}
grid_knn = GridSearchCV(pipe_knn, param_knn, cv=LeaveOneOut(), scoring='accuracy', n_jobs=-1)
grid_knn.fit(X_processed, y)
print(f"\n--- K-Nearest Neighbors ---")
print(f"Best Score: {grid_knn.best_score_:.4f}")
print(f"Best Params: {grid_knn.best_params_}")

# --- 3. Gaussian Naive Bayes (No Tuning) ---
# This model has no real hyperparameters, so we just get its score
pipe_gnb = Pipeline([
    # No scaling!
    ('gnb', GaussianNB())
])
grid_gnb = GridSearchCV(pipe_gnb, {}, cv=LeaveOneOut(), scoring='accuracy', n_jobs=-1) # Empty param grid
grid_gnb.fit(X_processed, y)
print(f"\n--- Gaussian Naive Bayes ---")
print(f"Best Score: {grid_gnb.best_score_:.4f}")


# --- 4. Neural Network (WARNING) ---
# This is *very* likely to overfit, but here is how you'd do it.
# We test a *tiny* network.

# --- Encode your labels ---
le = LabelEncoder()
# This converts ['Happiness', 'Anger', 'Sadness'] into [1, 0, 2]
y_numeric = le.fit_transform(y)

print("Label mapping (String -> Number):")
print(dict(zip(le.classes_, le.transform(le.classes_))))

pipe_nn = Pipeline([
    ('scale', StandardScaler()),
    ('nn', MLPClassifier(max_iter=1000, early_stopping=True, random_state=42))
])
# We tune the size of the single hidden layer
param_nn = {
    'nn__hidden_layer_sizes': [(10,), (20,), (10, 5)] # (10,) = one layer of 10 neurons
}
grid_nn = GridSearchCV(pipe_nn, param_nn, cv=LeaveOneOut(), scoring='accuracy', n_jobs=-1)
grid_nn.fit(X_processed, y_numeric)
print(f"\n--- Neural Network (Warning: High Overfit Risk) ---")
print(f"Best Score: {grid_nn.best_score_:.4f}")
print(f"Best Params: {grid_nn.best_params_}")


# --- 5. RandomForest (Tuning) ---
# We can tune this too, for a fair comparison
pipe_rf = Pipeline([
    # No scaling needed
    ('rf', RandomForestClassifier(random_state=42))
])
param_rf = {
    'rf__n_estimators': [50, 100],
    'rf__max_depth': [3, 5, None]
}
grid_rf = GridSearchCV(pipe_rf, param_rf, cv=LeaveOneOut(), scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_processed, y)
print(f"\n--- Random Forest ---")
print(f"Best Score: {grid_rf.best_score_:.4f}")
print(f"Best Params: {grid_rf.best_params_}")


--- SVM ---
Best Score: 0.5741
Best Params: {'svm__C': 0.1, 'svm__kernel': 'linear'}

--- K-Nearest Neighbors ---
Best Score: 0.4074
Best Params: {'knn__n_neighbors': 5}

--- Gaussian Naive Bayes ---
Best Score: 0.4444
Label mapping (String -> Number):
{'Anger': 0, 'Happiness': 1, 'Sadness': 2}

--- Neural Network (Warning: High Overfit Risk) ---
Best Score: 0.4444
Best Params: {'nn__hidden_layer_sizes': (10, 5)}

--- Random Forest ---
Best Score: 0.5741
Best Params: {'rf__max_depth': None, 'rf__n_estimators': 100}


In [78]:
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Define the cross-validation strategy
loocv = LeaveOneOut()

def get_svm_importances(fitted_estimator):
    """
    Returns a 1D array of importances from a fitted SVM
    by taking the L2 norm (Euclidean distance) of the coefficients
    across the classes.
    """
    # 1. Access the fitted SVC model from the pipeline
    svm_model = fitted_estimator.named_steps['svm']
    
    # 2. Get the 2D coefficient matrix (shape [n_classes, n_features])
    coefs = svm_model.coef_
    
    # 3. Calculate the L2 norm across the class rows (axis=0)
    # This results in a 1D array of shape [n_features]
    importances = np.linalg.norm(coefs, axis=0)
    
    return importances

def get_lr_importances(fitted_estimator):
    """
    Returns a 1D array of importances from a fitted LogisticRegression
    by taking the L2 norm (Euclidean distance) of the coefficients
    across the classes.
    """
    # 1. Access the fitted LogisticRegression model from the pipeline
    lr_model = fitted_estimator.named_steps['lr'] # <-- Accesses the 'lr' step
    
    # 2. Get the 2D coefficient matrix (shape [n_classes, n_features])
    coefs = lr_model.coef_
    
    # 3. Calculate the L2 norm across the class rows (axis=0)
    importances = np.linalg.norm(coefs, axis=0)
    
    return importances

# --- 1. RandomForest (Your Original) ---
print("Running RFECV with RandomForest...")
rf_model = RandomForestClassifier(random_state=42)
rfecv_rf = RFECV(estimator=rf_model, step=1, cv=loocv, scoring='accuracy', n_jobs=-1)
rfecv_rf.fit(X_processed, y)

print(f"Optimal features for RF: {rfecv_rf.n_features_}")

# --- 2. SVM (Must be Linear) ---
# Non-linear kernels (like 'rbf') do not have a .coef_ attribute
# We use a Pipeline to scale data *inside* the CV loop, which is best practice.
print("Running RFECV with Linear SVM...")
svc_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('svm', SVC(kernel='linear', C=1, random_state=42)) 
])

rfecv_svm = RFECV(
    estimator=svc_pipeline, 
    step=1, 
    cv=loocv, 
    scoring='accuracy', 
    n_jobs=-1,
    importance_getter=get_svm_importances 
)
rfecv_svm.fit(X_processed, y)
print(f"Optimal features for SVM: {rfecv_svm.n_features_}")

# --- 3. Logistic Regression (as a simple/stable NN) ---
print("Running RFECV with Logistic Regression...")
lr_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('lr', LogisticRegression(multi_class='ovr', solver='liblinear', random_state=42))
])
rfecv_lr = RFECV(
    estimator=lr_pipeline, 
    step=1, 
    cv=loocv, 
    scoring='accuracy', 
    n_jobs=-1,
    importance_getter=get_lr_importances 
)
rfecv_lr.fit(X_processed, y)

print(f"Optimal features for LogReg: {rfecv_lr.n_features_}")

Running RFECV with RandomForest...
Optimal features for RF: 1
Running RFECV with Linear SVM...
Optimal features for SVM: 1
Running RFECV with Logistic Regression...
Optimal features for LogReg: 4


In [111]:
import pickle
classifiers = pickle.load(open(r'C:\Users\Mario Tokumori\Documents\TUDarmstadt\6. Semester\Thesis\Alexithymia\classifiers_.pkl', 'rb' ))

In [112]:
pid_to_print = 1

# --- Check if participant data exists ---
if pid_to_print not in classifiers:
    print(f"Error: Participant {pid_to_print} not found in the results.")
else:
    participant_data = classifiers[pid_to_print]
    
    print(f"--- 📊 Results for Participant {pid_to_print} ---")
    
    # Iterate over each condition (e.g., 'audio+image')
    for condition, models in participant_data.items():
        print(f"\n  Condition: {condition}")
        print("  " + "="*35)
        
        # Iterate over each model entry
        for model_name, model_obj in models.items():
            
            # --- Handle GridSearchCV objects ---
            if "grid_" in model_name:
                try:
                    print(f"    🔹 {model_name}:")
                    print(f"        Best Score:  {model_obj.best_score_:.4f}")
                    print(f"        Best Params: {model_obj.best_params_}")
                except AttributeError:
                    print(f"    🔹 {model_name}: (Object is not a fitted GridSearchCV)")
                except Exception as e:
                    print(f"    🔹 {model_name}: Error printing results - {e}")
            
            # --- Handle RFECV objects ---
            elif "rfecv_" in model_name:
                try:
                    print(f"    🔸 {model_name}:")
                    print(f"        Optimal Features: {model_obj.n_features_}")
                    
                    # Get the score that corresponds to the optimal number of features
                    # This assumes 'min_features_to_select=1' and 'step=1'
                    score_index = model_obj.n_features_ - 1 
                    best_score = model_obj.cv_results_['mean_test_score'][score_index]
                    
                    print(f"        Score at {model_obj.n_features_} features: {best_score:.4f}")
                except AttributeError:
                     print(f"    🔸 {model_name}: (Object is not a fitted RFECV)")
                except Exception as e:
                    print(f"    🔸 {model_name}: Error printing results - {e}")

            # --- Handle LabelEncoder ---
            elif model_name == 'label_encoder':
                try:
                    print(f"    🏷️ {model_name}:")
                    mapping = dict(zip(model_obj.classes_, model_obj.transform(model_obj.classes_)))
                    print(f"        Mapping: {mapping}")
                except Exception:
                    pass # Ignore if it's not a fitted encoder
            
            # --- Other objects (like functions) are skipped ---
            else:
                pass

--- 📊 Results for Participant 1 ---

  Condition: audio+image
    🔹 grid_svm:
        Best Score:  0.6111
        Best Params: {'svm__C': 10, 'svm__kernel': 'rbf'}
    🔹 grid_knn:
        Best Score:  0.5741
        Best Params: {'knn__n_neighbors': 3}
    🔹 grid_gnb:
        Best Score:  0.6667
        Best Params: {}
    🏷️ label_encoder:
        Mapping: {'Anger': 0, 'Happiness': 1, 'Sadness': 2}
    🔹 grid_nn:
        Best Score:  0.3333
        Best Params: {'nn__hidden_layer_sizes': (10, 5)}
    🔹 grid_rf:
        Best Score:  0.7037
        Best Params: {'rf__max_depth': 3, 'rf__n_estimators': 100}
    🔸 rfecv_rf:
        Optimal Features: 33
        Score at 33 features: 0.7222
    🔸 rfecv_svm:
        Optimal Features: 16
        Score at 16 features: 0.5741
    🔸 rfecv_lr:
        Optimal Features: 31
        Score at 31 features: 0.6296

  Condition: image-only
    🔹 grid_svm:
        Best Score:  0.6296
        Best Params: {'svm__C': 1, 'svm__kernel': 'linear'}
    🔹 grid_